<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/education_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model Performace History

1.   Cosidering life expectancy as input
      
      1. *Training using max no of countries:*\
          **MSE**: 1.421960711479187
          
      2. *Training using min no of countries:*\
          **MSE**: 1.6800711154937744

      3. *Training using only common countries:*\
          **MSE**: 1.5712116956710815

2.   Not cosidering life expectancy as input
      1. *Trained using max no of countries:*\
          **MSE**: 1.9395439624786377

      2. *Trained using min no of countries*:\
          **MSE**: 2.161480665206909

      3. *Trained using only common countries*:\
          **MSE**: 2.10016131401062



In [7]:
!rm -rf education-impact
!rm education-impact

rm: cannot remove 'education-impact': No such file or directory


In [8]:
!git clone https://github.com/jaya-shankar/education-impact.git


Cloning into 'education-impact'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 279 (delta 135), reused 132 (delta 38), pack-reused 0
Receiving objects: 100% (279/279), 1.83 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [9]:
!pip install tensorflow_decision_forests
!pip install wurlitzer
!pip install seaborn

     |████████████████████████████████| 17.1 MB 186 kB/s 


In [10]:
import pandas as pd
import numpy as np
import math
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes

In [11]:
#@title Default title text
root = "education-impact/" 
datasets_path = {
                    "infant_mortality"              :  root+ "datasets/Infant_Mortality_Rate.csv",
                    "child_mortality"               :  root+ "datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv",
                    "children_per_woman"            :  root+ "datasets/children_per_woman_total_fertility.csv",
                    "co2_emissions"                 :  root+"datasets/co2_emissions_tonnes_per_person.csv",
                    "population"                    :  root+ "datasets/converted_pop.csv",
                    "food_supply"                   :  root+ "datasets/food_supply_kilocalories_per_person_and_day.csv",
                    "gdp_per_captia"                :  root+ "datasets/gdp_per_capita_yearly_growth.csv",
                    "Avg_daily_income_ppp"          :  root+ "datasets/mincpcap_cppp.csv",
                    "gini_index"                    :  root+ "datasets/gini.csv",
                    "life_expectancy"               :  root+ "datasets/life_expectancy_years.csv",
                    "malnutrition"                  :  root+ "datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv",
                    "poverty_index"                 :  root+ "datasets/mincpcap_cppp.csv",
                    "maternal_mortality"            :  root+ "datasets/mmr_who.csv",
                    "people_in_poverty"             :  root+ "datasets/number_of_people_in_poverty.csv",
                    "primary_completion"            :  root+ "datasets/primary_school_completion_percent_of_girls.csv",
                    "ratio_b/g_in_primary"          :  root+ "datasets/ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv",
                    "wcde-25--34"                   :  root+ "datasets/wcde-25--34.csv",
                    "wcde-Incomplete_Primary"       :  root+ "datasets/wcde-Incomplete Primary.csv",
                    "wcde-Lower_Secondary"          :  root+ "datasets/wcde-Lower Secondary.csv",
                    "wcde-Post_Secondary"           :  root+ "datasets/wcde-Post Secondary.csv",
                    "wcde_female-Incomplete_Primary":  root+ "datasets/wcde_female-Incomplete Primary.csv",
                    "wcde_female-Lower_Secondary"   :  root+ "datasets/wcde_female-Lower Secondary.csv",
                    "wcde_female-Post_Secondary"    :  root+ "datasets/wcde_female-Post Secondary.csv",
                    "sex-ratio"                     :  root+ "datasets/sex-ratio.csv",
                }

In [12]:
datasets = [
            "infant_mortality",
            # "life_expectancy",
            #"child_mortality",
            #"co2_emissions",
            "Avg_daily_income_ppp",
            "primary_completion",
            "wcde-Lower_Secondary",
            "population",
            #"wcde_female-Lower_Secondary",
            "sex-ratio",
            "gdp_per_captia",
            "wcde-Post_Secondary"
            ]

In [30]:
PREDICT_FUTURE  = 10
OUTPUTS         = ['wcde-Incomplete_Primary']
                   

In [23]:
# to find out how many countries each dataset has
countries_count      = None
least_dataset_path   = None
for dataset in datasets:
  df = pd.read_csv(datasets_path[dataset])
  count = len(set(df.Country.unique()))
  if not countries_count:
    countries_count = count
    least_dataset_path   = datasets_path[dataset]
  elif countries_count > count:
    countries_count = count
    least_dataset_path   = datasets_path[dataset]
  print(f"{'Factor: ' + dataset:<30} count: {count}")
print(f"{'To use: ' + least_dataset_path:<30} count: {countries_count}")

Factor: infant_mortality       count: 266
Factor: Avg_daily_income_ppp   count: 195
Factor: primary_completion     count: 195
Factor: wcde-Lower_Secondary   count: 202
Factor: population             count: 197
Factor: sex-ratio              count: 212
Factor: gdp_per_captia         count: 221
Factor: wcde-Post_Secondary    count: 202
To use: education-impact/datasets/mincpcap_cppp.csv count: 195


In [24]:
common_countries = set()
for dataset in datasets:
  countries_list = list(pd.read_csv(datasets_path[dataset]).Country)
  countries_list = set(map(lambda x: x.lower(), countries_list))
  if common_countries == set():
    common_countries = countries_list
  else:
    common_countries = common_countries.intersection(countries_list)
len(common_countries)

145

In [25]:
# creating a list of all countries & years
countries = list(common_countries)
years     = [y for y in range(1960,2015-PREDICT_FUTURE+1)]

In [26]:
len(countries)

145

In [27]:
keys=[]
for y in years:
  for c in countries:
    keys.append((c,str(y)))

In [28]:
big_dic = {k : [] for k in keys}
for dataset in datasets:
  df = pd.read_csv(datasets_path[dataset])
  df["Country"] = df["Country"].str.lower()
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][k[1]])
    except:
      big_dic[k].append(np.NaN)
 

In [31]:
for output_path in OUTPUTS:
  df = pd.read_csv(datasets_path[output_path])
  df["Country"] = df["Country"].str.lower()
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][str(int(k[1])+PREDICT_FUTURE)])
    except:
      big_dic[k].append(np.NaN)

In [32]:
columns = [k for k in datasets ]
output_columns = ["o_"+o for o in OUTPUTS]
columns.extend(output_columns)

In [33]:
input_df = pd.DataFrame.from_dict(big_dic,orient='index', columns = columns)
output_df = input_df[["o_"+o for o in OUTPUTS]]
input_df.drop(labels=["o_"+o for o in OUTPUTS], axis = 1, inplace=True)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(input_df, output_df, test_size=0.30, random_state=43)

In [35]:
X_train.isna().sum()

infant_mortality         475
Avg_daily_income_ppp       0
primary_completion      3182
wcde-Lower_Secondary       0
population                 0
sex-ratio                  0
gdp_per_captia             0
wcde-Post_Secondary        0
dtype: int64

In [36]:
y_train.isna().sum()

o_wcde-Incomplete_Primary    0
dtype: int64

In [37]:
y_train

,o_wcde-Incomplete_Primary
"(georgia, 1963)",0.96
"(greece, 1971)",1.96
"(azerbaijan, 1994)",0.20
"(eritrea, 1973)",0.00
"(equatorial guinea, 1986)",23.28
...,...
"(honduras, 2002)",31.38
"(ethiopia, 1976)",86.36
"(armenia, 1975)",0.30
"(solomon islands, 1983)",35.60


In [40]:
combine_dfs("o_"+OUTPUTS[0],X_test,y_test)

,infant_mortality,Avg_daily_income_ppp,primary_completion,wcde-Lower_Secondary,population,sex-ratio,gdp_per_captia,wcde-Post_Secondary,o_wcde-Incomplete_Primary
"(cote d'ivoire, 1985)",106.0,8.63,36.5,30.10,9920000,102.822771,0.469,2.50,64.20
"(sri lanka, 1980)",36.8,3.27,NaN,48.00,15000000,104.337239,3.670,26.10,17.90
"(malaysia, 1976)",32.4,8.95,NaN,53.82,12500000,105.784594,9.890,22.94,13.62
"(saudi arabia, 1996)",23.7,25.60,NaN,42.94,19000000,105.003345,-1.840,42.50,6.44
"(nepal, 1989)",101.6,1.73,NaN,30.34,18400000,104.447811,-0.252,8.68,45.06
...,...,...,...,...,...,...,...,...,...
"(tunisia, 1980)",68.9,5.43,47.9,62.10,6370000,105.580207,4.700,4.90,23.80
"(chile, 1971)",66.1,12.00,87.7,55.60,9940000,104.117952,6.990,24.00,12.62
"(ethiopia, 1997)",96.6,2.23,NaN,17.84,60700000,105.714366,0.122,2.68,70.14
"(cameroon, 1960)",163.6,2.54,NaN,26.30,5180000,102.599943,1.160,0.80,54.90


In [39]:
def combine_dfs(label,X,y):
  frames      = [X,y[label]]
  le_model_df = pd.concat(frames,axis=1)
  le_model_df.dropna(subset=[label],inplace=True)
  return le_model_df

In [41]:
for output in OUTPUTS:
  train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(combine_dfs("o_"+output,X_train,y_train), label="o_"+output, task=tfdf.keras.Task.REGRESSION)
  model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
  
  model.fit(x=train_ds)
  # Convert it to a TensorFlow dataset
  test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(combine_dfs("o_"+output,X_train,y_train), label="o_"+output, task=tfdf.keras.Task.REGRESSION)

  # Evaluate the model
  model.compile(metrics=["mse"])
  # Evaluate the model on the test dataset.
  evaluation = model.evaluate(test_ds, return_dict=True)
  print(output.upper())
  print(evaluation)
  print()
  print(f"MSE: {evaluation['mse']}")
  print(f"RMSE: {math.sqrt(evaluation['mse'])}")
  print()

73/73 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - mse: 2.9456
WCDE-INCOMPLETE_PRIMARY
{'loss': 0.0, 'mse': 2.945612668991089}

MSE: 2.945612668991089
RMSE: 1.7162787270694375



In [42]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)

In [43]:
# %set_cell_height 300

model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (8):
	Avg_daily_income_ppp
	gdp_per_captia
	infant_mortality
	population
	primary_completion
	sex-ratio
	wcde-Lower_Secondary
	wcde-Post_Secondary

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.              "__LABEL" 11.293061 ################
    2.       "gdp_per_captia" 10.298038 ##############
    3.   "primary_completion"  9.754294 #############
    4.           "population"  7.931297 ##########
    5.            "sex-ratio"  5.689731 ######
    6.     "infant_mortality"  4.365876 ####
    7. "Avg_daily_income_ppp"  4.301345 ####
    8. "wcde-Lower_Secondary"  4.247263 ####
    9.  "wcde-Post_Secondary"  1.463571 

Var

# RNN Algorithm

from the above output
- **malnutrition & people in povery** have least no of countries
- **infant mortality & gdp per captia** have highest no of countries

*Doubt:* Does having more data for one factor will make the decision tree bias?


###Steps
1. create a csv file such that each row contains all values of particular year & country present
2. the output for each row is year + 40 years corresponding value 
    1. **outputs** - life expectany, education level, gdp




From above output
- if we dont drop any rows our table size = 4256 entries
- if we drop rows containing any if all of outputs missing then our table size = 3039 entries
- if we drop rows containing any one of output missing then our table size = 1745 entries

so, I think its is better to go with second choice and build different models, but not sure it will not effect performance of the model


now we have the dataframe containing both inputs and ouputs,our next step is
1. split the data into train & test data
  1. try to split data based on continents to reduce bias
2. build DF model using tensorflow
3. check the accuracy of the model